In [1]:
from __future__ import print_function
import os, sys, warnings, json
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import scipy, sklearn
import matplotlib.pyplot as plt
import librosa, librosa.display

In [2]:
sr = 22050 
n_fft = 512 
hop_length = 512 
frame_length = 512 

map_dummy = {0: "a",1: "b",2: "c",3: "d",4: "e",5: "f",6: "g",7: "h",8: "i",9: "j",10:"k",11:"l"}
bandIndex2chord = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]


In [6]:
fp = "../data/test/3/feature.json"
with open(fp) as f:
    feature = json.load(f)
print(feature.keys())

dict_keys(['chroma_stft', 'chroma_cqt', 'chroma_cens', 'rms', 'spectral_centroid', 'spectral_bandwidth', 'spectral_contrast', 'spectral_flatness', 'spectral_rolloff', 'poly_features', 'tonnetz', 'zero_crossing_rate'])


In [3]:
def encode(string):
    counter = 1
    result = ""
    previousLetter = string[0]
    if len(string)==1:
        return str(1) + string[0]
    for i in range(1,len(string),1):
        if not string[i] == previousLetter:
            result += str(counter) + string[i-1]
            previousLetter = string[i]
            counter = 1

        else:
            counter += 1
        if i == len(string)-1:
                result += str(counter) + string[i]
    return result


In [7]:
maxPitch = np.argmax(np.array(feature["chroma_cqt"]), axis=0)
checkLength = 40
startFrame, endFrame = [], []

c = 0
for i in range(len(maxPitch)):
    startFrame.append(np.argmax(np.bincount(maxPitch[i:i+checkLength])))

map_results = []
for i in startFrame:
    map_results.append(map_dummy[i])
map_results    
    

result = encode(''.join(map_results))
print(result)
result = result.replace("a", " ").replace("b", " ").replace("c", " ").replace("d", " ").replace("e", " ").replace("f", " ").replace("g", " ").replace("h", " ").replace("i", " ").replace("j", " ") .replace("k", " ").replace("l", " ")
result = [int(i) for i in result.split(" ")[:-1]]
SUM = 0
startFrameGT = [0]
for i in result:
    SUM += i
    startFrameGT.append(SUM)
startFrameGT = np.array(startFrameGT)
len(startFrameGT), startFrameGT[:5]

5b78a23i40f21h42a4c64g23l13f57g4a27d53g21a7d129g9e95c79l5g42e46f82g96c89l46e37f74g11e94c87l48e42f65g26e82c89l44e47f42g43i41j47c243b31g79c6g4l75g31e43l3e10l5g3b15g49b28g66c49l58g55e19l2i16g31b82i6e47c171b196g91b152g7b49g25b54g14b110g44b28g155b45i59e26j90c102e44j28i44b50e73c30i59b22e16i6e29i52e18h6j3e26j3i89b84c11l6g7l1g46l30g47e1i13e9i6e81b11g90c44l8g6l10g7l1g24l53e18i60g10b45g105b73g18l46g51b2g155b2g33i97e95c104e21j11e38i42b47e80c81b43i81e26i3e3j1e1j25e1j92b7g78c42l56g79e75g12b5g86c58l28g34e9l52e41b88i41c175b194g5l15g3l13g6l8g6l11g8j85e90c70h14g99e82j47l48g49b2e8b1e4b17g10l1g9l12g2l44g96e83c69h18g53e22l81j20e67g11l12c59i7b10i7b38g55b61c49g61c32e17l42e1b64g9b1g3i77c14g7l20g1l1g2l13g10l39c43e20i4l3i16l47g52i8j26i44j152b12i104b76c59l42g7l7e2l28e8i8e1i16e5i3e33g24b4g7b17g90c29l3g2l54g36e4i3l22i16l2j7b9j37b28i29j20i48c65b27i88b99g70c8l56g26l10i79e58b29g76c9l34j58l1e3l34e40l2i41b11j78i9g40c178b91g77c55l47g76e5b11i


(332, array([  0,   5,  83, 106, 146]))

In [8]:
argMaxFourth = []
for arr in np.array(feature["chroma_cqt"]).T[:]:
    argMaxFourth.append(np.argsort(-arr)[:4])
argMaxFourth = np.array(argMaxFourth)
argMaxFourth

array([[ 1,  0,  2, 10],
       [ 1,  0,  2,  9],
       [ 1,  0,  2,  9],
       ...,
       [11,  8,  5,  1],
       [ 8,  1,  5, 11],
       [ 8,  1,  5, 11]], dtype=int64)

In [4]:
mainChord = {
    0 : {4:[10,11], 3:[10,11], 10:[3]}, 
    1 : {5:[11,0], 4:[11,0], 11:[4]},
    2 : {6:[0,1], 5:[0,1], 0:[5]},
    3 : {7:[1,2], 6:[1,2], 1:[6]},
    4 : {8:[2,3], 7:[2,3], 2:[7]},
    5 : {9:[3,4], 8:[3,4], 3:[8]},
    6 : {10:[4,5], 9:[4,5], 4:[9]},
    7 : {11:[5,6], 10:[5,6], 5:[10]},
    8 : {0:[6,7], 11:[6,7], 6:[11]},
    9 : {1:[7,8], 0:[7,8], 7:[0]},
    10 : {2:[8,9], 1:[8,9], 8:[1]},
    11 : {3:[9,10], 2:[9,10], 9:[2]}
}

In [10]:
testChord = []
testChortMajMin = []
testChordCombine = []
rootAllChord = []
# startFrameGT = result
for i in range(len(startFrameGT[:-1])):
    print(f"Seg{i:>5}:", end=" ")
    root_first = np.argmax(np.bincount(argMaxFourth[startFrameGT[i]:startFrameGT[i+1], 0]))
    print(root_first, end=" ")
    arr = argMaxFourth[startFrameGT[i]:startFrameGT[i+1], 1]
    removeFirst = np.delete(arr, 
                            np.where(arr == root_first))
    root_secon = np.argmax(np.bincount(removeFirst))
    print(root_secon, end=" ")
    arr = argMaxFourth[startFrameGT[i]:startFrameGT[i+1], 2]
    removeSecon = np.delete(arr, 
                            np.where(arr == root_first))
    removeSecon = np.delete(removeSecon, 
                            np.where(removeSecon == root_secon))
    root_third = np.argmax(np.bincount(removeSecon))
    print(root_third, end=" ")
    arr = argMaxFourth[startFrameGT[i]:startFrameGT[i+1], 3]
    removeThird = np.delete(arr, 
                            np.where(arr == root_first))
    removeThird = np.delete(removeThird, 
                            np.where(removeThird == root_secon))
    removeThird = np.delete(removeThird, 
                            np.where(removeThird == root_third))
    if len(np.bincount(removeThird))==0:
        root_forth = root_third
    else:
        root_forth = np.argmax(np.bincount(removeThird))
    print(root_forth, end=" ")
    
    print()

    testChord.append(bandIndex2chord[root_first])
    
    
    if   root_secon in mainChord[root_first].keys():
        if len(mainChord[root_first][root_secon]) == 1:
            if mainChord[root_first][root_secon][0] in [root_third, root_forth]:
                testChortMajMin.append("min7")
            else:
                testChortMajMin.append("7")
        else:
            if ((root_secon-root_first)%12)%4 == 0:
                if ((root_third - root_first)%12)%4 == 2:
                    testChortMajMin.append("7")
                elif ((root_third - root_first)%12)%4 == 3:
                    testChortMajMin.append("maj7")
                else:
                    testChortMajMin.append("maj")
            elif ((root_secon-root_first)%12)%3 == 0:
                if ((root_secon-root_first)%12)%3 == 1:
                    testChortMajMin.append("min7")
                elif ((root_secon-root_first)%12)%3 == 2:
                    testChortMajMin.append("7")
                else:
                    testChortMajMin.append("min")
    else:
        testChortMajMin.append("maj")
    rootAllChord.append([root_first, root_secon, root_third, root_forth])
    testChordCombine.append(f"{testChord[-1]}:{testChortMajMin[-1]}")

Seg    0: 1 0 2 9 
Seg    1: 0 3 1 11 
Seg    2: 0 11 1 2 
Seg    3: 5 8 6 3 
Seg    4: 5 8 6 7 
Seg    5: 0 11 3 2 
Seg    6: 0 2 1 3 
Seg    7: 6 0 1 3 
Seg    8: 6 10 1 0 
Seg    9: 11 8 0 9 
Seg   10: 6 8 5 1 
Seg   11: 6 8 7 10 
Seg   12: 0 2 3 1 
Seg   13: 6 5 0 1 
Seg   14: 6 11 2 1 
Seg   15: 0 8 11 2 
Seg   16: 6 0 1 3 
Seg   17: 6 1 0 3 
Seg   18: 2 6 9 1 
Seg   19: 11 2 6 5 
Seg   20: 11 2 6 1 
Seg   21: 4 11 2 6 
Seg   22: 5 11 8 0 
Seg   23: 6 1 9 5 
Seg   24: 2 9 6 1 
Seg   25: 11 2 6 9 
Seg   26: 4 11 2 5 
Seg   27: 4 11 6 0 
Seg   28: 6 1 9 5 
Seg   29: 6 1 9 4 
Seg   30: 2 9 6 1 
Seg   31: 11 2 6 9 
Seg   32: 4 11 2 1 
Seg   33: 5 4 8 11 
Seg   34: 6 1 9 5 
Seg   35: 6 1 4 5 
Seg   36: 2 9 6 1 
Seg   37: 11 2 6 9 
Seg   38: 4 11 2 1 
Seg   39: 5 4 8 1 
Seg   40: 6 1 5 9 
Seg   41: 8 6 11 7 
Seg   42: 9 8 11 10 
Seg   43: 2 9 6 1 
Seg   44: 1 8 6 5 
Seg   45: 1 6 5 8 
Seg   46: 2 1 9 3 
Seg   47: 2 4 6 9 
Seg   48: 11 2 6 9 
Seg   49: 6 11 2 3 
Seg   50: 6 11 2 3 
Seg  

In [51]:
for c, i, k in zip(range(len(testChord)), testChordCombine, rootAllChord):
    print(c, i, k[1:])

0 C:maj [1, 2, 3]
1 A:maj [11, 0, 10]
2 A#:maj [9, 0, 7]
3 A#:maj [4, 2, 9]
4 C#:maj [0, 2, 5]
5 D:maj [3, 4, 11]
6 G:maj [1, 2, 0]
7 A:maj [10, 7, 8]
8 A:maj [8, 7, 10]
9 E:maj [8, 0, 5]
10 D#:maj [2, 0, 1]
11 C#:maj [0, 4, 3]
12 C:maj [1, 9, 4]
13 C:maj [9, 5, 1]
14 C#:min [4, 5, 9]
15 A:7 [7, 4, 6]
16 D:maj [7, 9, 6]
17 G:maj [2, 9, 8]
18 G:maj [8, 9, 6]
19 C:maj [7, 8, 9]
20 G:maj [6, 9, 4]
21 A:7 [7, 6, 8]
22 D:maj7 [6, 9, 7]
23 D:maj [9, 7, 6]
24 G:maj [2, 6, 9]
25 A:maj [8, 7, 6]
26 D:maj [7, 9, 6]
27 D:maj [7, 9, 6]
28 G:maj [6, 8, 9]
29 G:maj [8, 9, 6]
30 C:maj [7, 6, 8]
31 G:maj [0, 6, 2]
32 A:7 [7, 6, 4]
33 D:maj [7, 8, 6]
34 G:maj [8, 9, 6]
35 C:maj [7, 6, 9]
36 E:min [7, 8, 9]
37 G:maj [8, 9, 6]
38 A:min [0, 7, 8]
39 D:maj [6, 7, 9]
40 G:maj [6, 9, 8]
41 B:maj [6, 9, 7]
42 A:maj [6, 8, 7]
43 B:7 [9, 6, 7]
44 B:maj [6, 9, 8]
45 A:maj [10, 6, 8]
46 A:7 [7, 8, 6]
47 D:maj [7, 9, 10]
48 G:maj [6, 9, 8]
49 C:maj [7, 8, 9]
50 C:maj [7, 9, 8]
51 A:7 [7, 8, 6]
52 D:maj [9, 7, 6]
5

In [40]:
for i in range(len(testChordCombine)):
    print(f"[{startFrameGT[i]*512/sr}, {startFrameGT[i+1]*512/sr}, {testChordCombine[i]}],\n")

[0.0, 1.253877551020408, F#:maj],

[1.253877551020408, 2.066575963718821, F#:maj],

[2.066575963718821, 7.291065759637188, F#:7],

[7.291065759637188, 7.75546485260771, F#:7],

[7.75546485260771, 9.218321995464853, F#:maj],

[9.218321995464853, 10.00780045351474, F#:maj],

[10.00780045351474, 10.47219954648526, F:maj],

[10.47219954648526, 11.493877551020407, D:maj7],

[11.493877551020407, 12.933514739229025, B:min],

[12.933514739229025, 13.02639455782313, B:maj7],

[13.02639455782313, 14.303492063492063, G:maj],

[14.303492063492063, 15.882448979591837, D:maj],

[15.882448979591837, 16.973786848072564, A:maj],

[16.973786848072564, 18.297324263038547, B:min],

[18.297324263038547, 18.45986394557823, B:min],

[18.45986394557823, 19.644081632653062, G:maj],

[19.644081632653062, 21.826757369614512, D:maj],

[21.826757369614512, 22.38403628117914, D:maj],

[22.38403628117914, 23.986213151927437, B:min],

[23.986213151927437, 25.14721088435374, G:maj],

[25.14721088435374, 26.42430839002

# 最後結果

In [14]:
RESULTS = []
for NUMBER in range(1,301):
    print(NUMBER)
    RESULTS.append([])
    fp = f"../data/test/{NUMBER}/feature.json"
    with open(fp) as f:
        feature = json.load(f)
    maxPitch = np.argmax(np.array(feature["chroma_cqt"]), axis=0)
    checkLength = 40
    startFrame = []
    for i in range(len(maxPitch)):
        startFrame.append(np.argmax(np.bincount(maxPitch[i:i+checkLength])))

    map_results = []
    for i in startFrame:
        map_results.append(map_dummy[i])
    result = encode(''.join(map_results))
    result = result.replace("a", " ").replace("b", " ").replace("c", " ").replace("d", " ").replace("e", " ").replace("f", " ").replace("g", " ").replace("h", " ").replace("i", " ").replace("j", " ") .replace("k", " ").replace("l", " ")
    result = [int(i) for i in result.split(" ")[:-1]]
    SUM = 0
    startFrameGT = [0]
    for i in result:
        SUM += i
        startFrameGT.append(SUM)
    startFrameGT = np.array(startFrameGT)
    argMaxFourth = []
    for arr in np.array(feature["chroma_cqt"]).T[:]:
        argMaxFourth.append(np.argsort(-arr)[:4])
    argMaxFourth = np.array(argMaxFourth)

    testChord = []
    testChortMajMin = []
    testChordCombine = []
    rootAllChord = []
    # startFrameGT = result
    for i in range(len(startFrameGT[:-1])):
#         print(f"Seg{i:>5}:", end=" ")
        root_first = np.argmax(np.bincount(argMaxFourth[startFrameGT[i]:startFrameGT[i+1], 0]))
#         print(root_first, end=" ")
        arr = argMaxFourth[startFrameGT[i]:startFrameGT[i+1], 1]
        removeFirst = np.delete(arr, 
                                np.where(arr == root_first))
        root_secon = np.argmax(np.bincount(removeFirst))
#         print(root_secon, end=" ")
        arr = argMaxFourth[startFrameGT[i]:startFrameGT[i+1], 2]
        removeSecon = np.delete(arr, 
                                np.where(arr == root_first))
        removeSecon = np.delete(removeSecon, 
                                np.where(removeSecon == root_secon))
        if len(np.bincount(removeSecon))==0:
            root_third = root_secon
        else: 
            root_third = np.argmax(np.bincount(removeSecon))
#         print(root_third, end=" ")
        arr = argMaxFourth[startFrameGT[i]:startFrameGT[i+1], 3]
        removeThird = np.delete(arr, 
                                np.where(arr == root_first))
        removeThird = np.delete(removeThird, 
                                np.where(removeThird == root_secon))
        removeThird = np.delete(removeThird, 
                                np.where(removeThird == root_third))
#         root_forth = np.argmax(np.bincount(removeThird))
        if len(np.bincount(removeThird))==0:
            root_forth = root_third
        else:
            root_forth = np.argmax(np.bincount(removeThird))
#         print(root_forth, end=" ")

#         print()

        testChord.append(bandIndex2chord[root_first])


        if   root_secon in mainChord[root_first].keys():
            if len(mainChord[root_first][root_secon]) == 1:
                if mainChord[root_first][root_secon][0] in [root_third, root_forth]:
                    testChortMajMin.append("min7")
                else:
                    testChortMajMin.append("7")
            else:
                if ((root_secon-root_first)%12)%4 == 0:
                    if ((root_third - root_first)%12)%4 == 2:
                        testChortMajMin.append("7")
                    elif ((root_third - root_first)%12)%4 == 3:
                        testChortMajMin.append("maj7")
                    else:
                        testChortMajMin.append("maj")
                elif ((root_secon-root_first)%12)%3 == 0:
                    if ((root_secon-root_first)%12)%3 == 1:
                        testChortMajMin.append("min7")
                    elif ((root_secon-root_first)%12)%3 == 2:
                        testChortMajMin.append("7")
                    else:
                        testChortMajMin.append("min")
        else:
            testChortMajMin.append("maj")
        rootAllChord.append([root_first, root_secon, root_third, root_forth])
        testChordCombine.append(f"{testChord[-1]}:{testChortMajMin[-1]}")
    for i in range(len(testChordCombine)):
        RESULTS[-1].append(f"[{startFrameGT[i]*512/sr}, {startFrameGT[i+1]*512/sr}, {testChordCombine[i]}],")
len(RESULTS )
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


300

In [15]:
RESULTS[0]

['[0.0, 1.486077097505669, A:maj],',
 '[1.486077097505669, 2.786394557823129, E:maj],',
 '[2.786394557823129, 4.20281179138322, F#:min7],',
 '[4.20281179138322, 4.388571428571429, F#:min7],',
 '[4.388571428571429, 4.690430839002268, F#:maj],',
 '[4.690430839002268, 5.828208616780046, E:maj],',
 '[5.828208616780046, 6.013968253968254, E:maj],',
 '[6.013968253968254, 6.03718820861678, A:maj],',
 '[6.03718820861678, 6.315827664399093, A:maj7],',
 '[6.315827664399093, 7.87156462585034, F#:min],',
 '[7.87156462585034, 9.427301587301587, E:maj],',
 '[9.427301587301587, 14.164172335600908, A:maj],',
 '[14.164172335600908, 15.27873015873016, E:maj7],',
 '[15.27873015873016, 15.975328798185942, E:maj],',
 '[15.975328798185942, 16.253968253968253, C#:maj],',
 '[16.253968253968253, 16.416507936507937, B:maj],',
 '[16.416507936507937, 16.92734693877551, F#:min],',
 '[16.92734693877551, 17.11310657596372, A:maj],',
 '[17.11310657596372, 17.322086167800453, A:maj],',
 '[17.322086167800453, 18.204444

In [ ]:
with open("")
for NUMBER in range(1,301):